In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
import keras

2023-10-30 10:06:51.401675: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-30 10:06:51.401704: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-30 10:06:51.401739: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-30 10:06:51.409236: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-30 10:06:52.142340: W tensorflow/compiler/

In [2]:
df = pd.read_csv('concrete_data.csv')

In [3]:
df.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [4]:
df.dtypes

Cement                float64
Blast Furnace Slag    float64
Fly Ash               float64
Water                 float64
Superplasticizer      float64
Coarse Aggregate      float64
Fine Aggregate        float64
Age                     int64
Strength              float64
dtype: object

In [5]:
df.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
df = (df-df.mean())/df.std()

In [7]:
df

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597,2.644123
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597,1.560663
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340,0.266498
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221,0.313188
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069,0.507732
...,...,...,...,...,...,...,...,...,...
1025,-0.045623,0.487998,0.564271,-0.092126,0.451190,-1.322363,-0.065861,-0.279597,0.506535
1026,0.392628,-0.856472,0.959602,0.675872,0.702285,-1.993711,0.496651,-0.279597,-0.277627
1027,-1.269472,0.759210,0.850222,0.521336,-0.017520,-1.035561,0.080068,-0.279597,-0.725377
1028,-1.168042,1.307430,-0.846733,-0.279443,0.852942,0.214537,0.191074,-0.279597,-0.182450


In [8]:
### Split the dataframe

X = df.drop(columns=['Strength'])
y = df['Strength']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=.3)

In [9]:
def regression_model():
    # create model
    model = Sequential()
    model.add(keras.Input(shape=(8,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [10]:
model = regression_model()

2023-10-30 10:06:53.049756: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [11]:
### Training the model 50 times. 

results_mse = []


for reg in range(1, 51):
    model.fit(X_train, y_train, epochs=50, verbose=2)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    results_mse.append(mse)

Epoch 1/50
23/23 - 0s - loss: 1.8850 - 401ms/epoch - 17ms/step
Epoch 2/50
23/23 - 0s - loss: 1.5527 - 24ms/epoch - 1ms/step
Epoch 3/50
23/23 - 0s - loss: 1.3155 - 25ms/epoch - 1ms/step
Epoch 4/50
23/23 - 0s - loss: 1.1474 - 25ms/epoch - 1ms/step
Epoch 5/50
23/23 - 0s - loss: 1.0220 - 23ms/epoch - 1ms/step
Epoch 6/50
23/23 - 0s - loss: 0.9223 - 24ms/epoch - 1ms/step
Epoch 7/50
23/23 - 0s - loss: 0.8413 - 24ms/epoch - 1ms/step
Epoch 8/50
23/23 - 0s - loss: 0.7721 - 23ms/epoch - 1ms/step
Epoch 9/50
23/23 - 0s - loss: 0.7127 - 24ms/epoch - 1ms/step
Epoch 10/50
23/23 - 0s - loss: 0.6612 - 21ms/epoch - 926us/step
Epoch 11/50
23/23 - 0s - loss: 0.6156 - 23ms/epoch - 988us/step
Epoch 12/50
23/23 - 0s - loss: 0.5760 - 29ms/epoch - 1ms/step
Epoch 13/50
23/23 - 0s - loss: 0.5440 - 24ms/epoch - 1ms/step
Epoch 14/50
23/23 - 0s - loss: 0.5153 - 24ms/epoch - 1ms/step
Epoch 15/50
23/23 - 0s - loss: 0.4906 - 24ms/epoch - 1ms/step
Epoch 16/50
23/23 - 0s - loss: 0.4692 - 25ms/epoch - 1ms/step
Epoch 17/50

23/23 - 0s - loss: 0.1266 - 24ms/epoch - 1ms/step
Epoch 30/50
23/23 - 0s - loss: 0.1266 - 22ms/epoch - 975us/step
Epoch 31/50
23/23 - 0s - loss: 0.1256 - 21ms/epoch - 907us/step
Epoch 32/50
23/23 - 0s - loss: 0.1255 - 19ms/epoch - 817us/step
Epoch 33/50
23/23 - 0s - loss: 0.1252 - 19ms/epoch - 815us/step
Epoch 34/50
23/23 - 0s - loss: 0.1253 - 21ms/epoch - 892us/step
Epoch 35/50
23/23 - 0s - loss: 0.1249 - 20ms/epoch - 849us/step
Epoch 36/50
23/23 - 0s - loss: 0.1248 - 20ms/epoch - 864us/step
Epoch 37/50
23/23 - 0s - loss: 0.1248 - 19ms/epoch - 822us/step
Epoch 38/50
23/23 - 0s - loss: 0.1247 - 20ms/epoch - 870us/step
Epoch 39/50
23/23 - 0s - loss: 0.1235 - 21ms/epoch - 913us/step
Epoch 40/50
23/23 - 0s - loss: 0.1238 - 20ms/epoch - 875us/step
Epoch 41/50
23/23 - 0s - loss: 0.1237 - 21ms/epoch - 899us/step
Epoch 42/50
23/23 - 0s - loss: 0.1233 - 21ms/epoch - 897us/step
Epoch 43/50
23/23 - 0s - loss: 0.1228 - 21ms/epoch - 920us/step
Epoch 44/50
23/23 - 0s - loss: 0.1225 - 21ms/epoch - 9

Epoch 6/50
23/23 - 0s - loss: 0.1071 - 22ms/epoch - 944us/step
Epoch 7/50
23/23 - 0s - loss: 0.1077 - 21ms/epoch - 902us/step
Epoch 8/50
23/23 - 0s - loss: 0.1076 - 22ms/epoch - 937us/step
Epoch 9/50
23/23 - 0s - loss: 0.1072 - 23ms/epoch - 1ms/step
Epoch 10/50
23/23 - 0s - loss: 0.1078 - 22ms/epoch - 978us/step
Epoch 11/50
23/23 - 0s - loss: 0.1075 - 21ms/epoch - 914us/step
Epoch 12/50
23/23 - 0s - loss: 0.1077 - 22ms/epoch - 970us/step
Epoch 13/50
23/23 - 0s - loss: 0.1076 - 22ms/epoch - 946us/step
Epoch 14/50
23/23 - 0s - loss: 0.1071 - 22ms/epoch - 954us/step
Epoch 15/50
23/23 - 0s - loss: 0.1070 - 21ms/epoch - 898us/step
Epoch 16/50
23/23 - 0s - loss: 0.1067 - 21ms/epoch - 915us/step
Epoch 17/50
23/23 - 0s - loss: 0.1069 - 23ms/epoch - 985us/step
Epoch 18/50
23/23 - 0s - loss: 0.1069 - 21ms/epoch - 918us/step
Epoch 19/50
23/23 - 0s - loss: 0.1066 - 21ms/epoch - 917us/step
Epoch 20/50
23/23 - 0s - loss: 0.1073 - 22ms/epoch - 966us/step
Epoch 21/50
23/23 - 0s - loss: 0.1064 - 21ms/e

Epoch 33/50
23/23 - 0s - loss: 0.1029 - 21ms/epoch - 934us/step
Epoch 34/50
23/23 - 0s - loss: 0.1028 - 20ms/epoch - 859us/step
Epoch 35/50
23/23 - 0s - loss: 0.1025 - 20ms/epoch - 873us/step
Epoch 36/50
23/23 - 0s - loss: 0.1028 - 22ms/epoch - 954us/step
Epoch 37/50
23/23 - 0s - loss: 0.1042 - 22ms/epoch - 940us/step
Epoch 38/50
23/23 - 0s - loss: 0.1032 - 20ms/epoch - 875us/step
Epoch 39/50
23/23 - 0s - loss: 0.1026 - 21ms/epoch - 909us/step
Epoch 40/50
23/23 - 0s - loss: 0.1023 - 19ms/epoch - 837us/step
Epoch 41/50
23/23 - 0s - loss: 0.1021 - 20ms/epoch - 887us/step
Epoch 42/50
23/23 - 0s - loss: 0.1027 - 20ms/epoch - 890us/step
Epoch 43/50
23/23 - 0s - loss: 0.1025 - 21ms/epoch - 913us/step
Epoch 44/50
23/23 - 0s - loss: 0.1023 - 21ms/epoch - 929us/step
Epoch 45/50
23/23 - 0s - loss: 0.1020 - 20ms/epoch - 860us/step
Epoch 46/50
23/23 - 0s - loss: 0.1028 - 21ms/epoch - 928us/step
Epoch 47/50
23/23 - 0s - loss: 0.1023 - 20ms/epoch - 868us/step
Epoch 48/50
23/23 - 0s - loss: 0.1025 - 

Epoch 10/50
23/23 - 0s - loss: 0.0981 - 21ms/epoch - 897us/step
Epoch 11/50
23/23 - 0s - loss: 0.0980 - 21ms/epoch - 903us/step
Epoch 12/50
23/23 - 0s - loss: 0.0985 - 20ms/epoch - 868us/step
Epoch 13/50
23/23 - 0s - loss: 0.0984 - 21ms/epoch - 921us/step
Epoch 14/50
23/23 - 0s - loss: 0.0982 - 21ms/epoch - 912us/step
Epoch 15/50
23/23 - 0s - loss: 0.0991 - 20ms/epoch - 875us/step
Epoch 16/50
23/23 - 0s - loss: 0.1000 - 21ms/epoch - 914us/step
Epoch 17/50
23/23 - 0s - loss: 0.0999 - 21ms/epoch - 927us/step
Epoch 18/50
23/23 - 0s - loss: 0.0987 - 21ms/epoch - 926us/step
Epoch 19/50
23/23 - 0s - loss: 0.0981 - 21ms/epoch - 932us/step
Epoch 20/50
23/23 - 0s - loss: 0.0981 - 20ms/epoch - 850us/step
Epoch 21/50
23/23 - 0s - loss: 0.0975 - 20ms/epoch - 887us/step
Epoch 22/50
23/23 - 0s - loss: 0.0975 - 20ms/epoch - 867us/step
Epoch 23/50
23/23 - 0s - loss: 0.0977 - 21ms/epoch - 914us/step
Epoch 24/50
23/23 - 0s - loss: 0.0980 - 20ms/epoch - 872us/step
Epoch 25/50
23/23 - 0s - loss: 0.0974 - 

...

Epoch 6/50
23/23 - 0s - loss: 0.0793 - 19ms/epoch - 842us/step
Epoch 7/50
23/23 - 0s - loss: 0.0796 - 22ms/epoch - 974us/step
Epoch 8/50
23/23 - 0s - loss: 0.0794 - 22ms/epoch - 941us/step
Epoch 9/50
23/23 - 0s - loss: 0.0794 - 20ms/epoch - 881us/step
Epoch 10/50
23/23 - 0s - loss: 0.0798 - 20ms/epoch - 888us/step
Epoch 11/50
23/23 - 0s - loss: 0.0803 - 19ms/epoch - 848us/step
Epoch 12/50
23/23 - 0s - loss: 0.0802 - 26ms/epoch - 1ms/step
Epoch 13/50
23/23 - 0s - loss: 0.0790 - 31ms/epoch - 1ms/step
Epoch 14/50
23/23 - 0s - loss: 0.0797 - 20ms/epoch - 870us/step
Epoch 15/50
23/23 - 0s - loss: 0.0793 - 20ms/epoch - 871us/step
Epoch 16/50
23/23 - 0s - loss: 0.0797 - 20ms/epoch - 867us/step
Epoch 17/50
23/23 - 0s - loss: 0.0791 - 22ms/epoch - 977us/step
Epoch 18/50
23/23 - 0s - loss: 0.0797 - 20ms/epoch - 889us/step
Epoch 19/50
23/23 - 0s - loss: 0.0816 - 20ms/epoch - 854us/step
Epoch 20/50
23/23 - 0s - loss: 0.0797 - 20ms/epoch - 876us/step
Epoch 21/50
23/23 - 0s - loss: 0.0795 - 21ms/epo

Epoch 34/50
23/23 - 0s - loss: 0.0792 - 21ms/epoch - 920us/step
Epoch 35/50
23/23 - 0s - loss: 0.0795 - 22ms/epoch - 944us/step
Epoch 36/50
23/23 - 0s - loss: 0.0806 - 24ms/epoch - 1ms/step
Epoch 37/50
23/23 - 0s - loss: 0.0796 - 21ms/epoch - 921us/step
Epoch 38/50
23/23 - 0s - loss: 0.0796 - 21ms/epoch - 925us/step
Epoch 39/50
23/23 - 0s - loss: 0.0793 - 21ms/epoch - 921us/step
Epoch 40/50
23/23 - 0s - loss: 0.0793 - 22ms/epoch - 963us/step
Epoch 41/50
23/23 - 0s - loss: 0.0792 - 20ms/epoch - 855us/step
Epoch 42/50
23/23 - 0s - loss: 0.0797 - 21ms/epoch - 929us/step
Epoch 43/50
23/23 - 0s - loss: 0.0797 - 22ms/epoch - 950us/step
Epoch 44/50
23/23 - 0s - loss: 0.0794 - 19ms/epoch - 838us/step
Epoch 45/50
23/23 - 0s - loss: 0.0802 - 21ms/epoch - 933us/step
Epoch 46/50
23/23 - 0s - loss: 0.0800 - 20ms/epoch - 885us/step
Epoch 47/50
23/23 - 0s - loss: 0.0796 - 22ms/epoch - 943us/step
Epoch 48/50
23/23 - 0s - loss: 0.0791 - 20ms/epoch - 869us/step
Epoch 49/50
23/23 - 0s - loss: 0.0797 - 21

23/23 - 0s - loss: 0.0794 - 23ms/epoch - 1ms/step
Epoch 11/50
23/23 - 0s - loss: 0.0785 - 23ms/epoch - 1ms/step
Epoch 12/50
23/23 - 0s - loss: 0.0788 - 22ms/epoch - 946us/step
Epoch 13/50
23/23 - 0s - loss: 0.0788 - 21ms/epoch - 918us/step
Epoch 14/50
23/23 - 0s - loss: 0.0788 - 22ms/epoch - 969us/step
Epoch 15/50
23/23 - 0s - loss: 0.0793 - 22ms/epoch - 961us/step
Epoch 16/50
23/23 - 0s - loss: 0.0792 - 21ms/epoch - 898us/step
Epoch 17/50
23/23 - 0s - loss: 0.0790 - 20ms/epoch - 886us/step
Epoch 18/50
23/23 - 0s - loss: 0.0790 - 22ms/epoch - 963us/step
Epoch 19/50
23/23 - 0s - loss: 0.0786 - 20ms/epoch - 860us/step
Epoch 20/50
23/23 - 0s - loss: 0.0788 - 22ms/epoch - 977us/step
Epoch 21/50
23/23 - 0s - loss: 0.0785 - 22ms/epoch - 950us/step
Epoch 22/50
23/23 - 0s - loss: 0.0791 - 22ms/epoch - 939us/step
Epoch 23/50
23/23 - 0s - loss: 0.0787 - 23ms/epoch - 981us/step
Epoch 24/50
23/23 - 0s - loss: 0.0785 - 22ms/epoch - 949us/step
Epoch 25/50
23/23 - 0s - loss: 0.0787 - 21ms/epoch - 926

Epoch 37/50
23/23 - 0s - loss: 0.0778 - 20ms/epoch - 871us/step
Epoch 38/50
23/23 - 0s - loss: 0.0785 - 20ms/epoch - 886us/step
Epoch 39/50
23/23 - 0s - loss: 0.0789 - 20ms/epoch - 867us/step
Epoch 40/50
23/23 - 0s - loss: 0.0782 - 20ms/epoch - 887us/step
Epoch 41/50
23/23 - 0s - loss: 0.0785 - 21ms/epoch - 914us/step
Epoch 42/50
23/23 - 0s - loss: 0.0780 - 21ms/epoch - 925us/step
Epoch 43/50
23/23 - 0s - loss: 0.0789 - 19ms/epoch - 831us/step
Epoch 44/50
23/23 - 0s - loss: 0.0790 - 21ms/epoch - 896us/step
Epoch 45/50
23/23 - 0s - loss: 0.0788 - 21ms/epoch - 922us/step
Epoch 46/50
23/23 - 0s - loss: 0.0782 - 20ms/epoch - 866us/step
Epoch 47/50
23/23 - 0s - loss: 0.0781 - 21ms/epoch - 903us/step
Epoch 48/50
23/23 - 0s - loss: 0.0783 - 20ms/epoch - 854us/step
Epoch 49/50
23/23 - 0s - loss: 0.0783 - 21ms/epoch - 898us/step
Epoch 50/50
23/23 - 0s - loss: 0.0781 - 18ms/epoch - 804us/step
10/10 [==============================] - 0s 863us/step
Epoch 1/50
23/23 - 0s - loss: 0.0777 - 21ms/epoch

Epoch 13/50
23/23 - 0s - loss: 0.0777 - 19ms/epoch - 841us/step
Epoch 14/50
23/23 - 0s - loss: 0.0773 - 19ms/epoch - 827us/step
Epoch 15/50
23/23 - 0s - loss: 0.0783 - 19ms/epoch - 839us/step
Epoch 16/50
23/23 - 0s - loss: 0.0781 - 19ms/epoch - 809us/step
Epoch 17/50
23/23 - 0s - loss: 0.0772 - 20ms/epoch - 884us/step
Epoch 18/50
23/23 - 0s - loss: 0.0775 - 20ms/epoch - 848us/step
Epoch 19/50
23/23 - 0s - loss: 0.0770 - 20ms/epoch - 879us/step
Epoch 20/50
23/23 - 0s - loss: 0.0772 - 21ms/epoch - 912us/step
Epoch 21/50
23/23 - 0s - loss: 0.0772 - 23ms/epoch - 983us/step
Epoch 22/50
23/23 - 0s - loss: 0.0778 - 22ms/epoch - 953us/step
Epoch 23/50
23/23 - 0s - loss: 0.0775 - 23ms/epoch - 1ms/step
Epoch 24/50
23/23 - 0s - loss: 0.0781 - 25ms/epoch - 1ms/step
Epoch 25/50
23/23 - 0s - loss: 0.0779 - 23ms/epoch - 987us/step
Epoch 26/50
23/23 - 0s - loss: 0.0775 - 22ms/epoch - 975us/step
Epoch 27/50
23/23 - 0s - loss: 0.0778 - 22ms/epoch - 953us/step
Epoch 28/50
23/23 - 0s - loss: 0.0778 - 22ms

In [12]:
results_mse

[0.20812973539610902,
 0.16377727644669077,
 0.15408276349297445,
 0.14142813143369928,
 0.13718688889100317,
 0.13566353633726,
 0.13451872091489525,
 0.1339214358382331,
 0.1365638302950858,
 0.13572630802998,
 0.13695915010401666,
 0.1373651884228614,
 0.1377298299972168,
 0.1372177112966329,
 0.13712282751221957,
 0.13656372065479713,
 0.13619979406106192,
 0.13685587838515137,
 0.13440613729719122,
 0.13555795418890149,
 0.1352071728882267,
 0.13591918883173174,
 0.13535462632786097,
 0.13594740151616175,
 0.13785444721496418,
 0.13736544824728725,
 0.13833805253489367,
 0.13990394135049347,
 0.13986541228399776,
 0.14302415455812928,
 0.14234982540960525,
 0.14351370735182079,
 0.14223153911324005,
 0.14018607247812528,
 0.1391677830641468,
 0.13936028101545758,
 0.13813425773943128,
 0.1383279742106302,
 0.1379246624827611,
 0.13747060509754777,
 0.13675991244772254,
 0.13641239168738456,
 0.13427182182962968,
 0.13480800837480028,
 0.13275370592591643,
 0.13242908250539187,
 0.

In [13]:
mse_df = pd.DataFrame(results_mse)

In [14]:
mse_df.head()

,0
0,0.208130
1,0.163777
2,0.154083
3,0.141428
4,0.137187


In [15]:
mse_df.rename(columns={0:'Mean Squared Error'}, inplace=True)

In [16]:
### Mean of the Mean Squared errors.

mse_df['Mean Squared Error'].mean()

0.1392200181765696

In [17]:
### Standard Deviation of the Mean Squared errors.

mse_df['Mean Squared Error'].std()

0.011248877044535406

B. Normalize the data (5 marks) 

Repeat Part A but use a normalized version of the data. Recall that one way to normalize the data is by subtracting the mean from the individual predictors and dividing by the standard deviation.

How does the mean of the mean squared errors compare to that from Step A?